In [1]:
import os
import re
import sys
import time
import implicit
import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import  tqdm

In [2]:
IS_LOCAL = False
HOME_DIR = ('/mnt/E/Projects/' if IS_LOCAL else '/home/Xetd71/') + 'Content-based-Neural-Recommender-Systems/'
os.environ['HOME_DIR'] = HOME_DIR

sys.path.append("../..")
from utils.prepare_data import zen
from utils.evaluate import test
from utils import tqdm_utils

DATA_DIR = f'{HOME_DIR}data/zen/'
WORKING_DIR = f'{HOME_DIR}models/mlp'
os.chdir(WORKING_DIR)

100%|██████████| 4349/4349 [00:13<00:00, 329.46it/s]


In [3]:
!pwd

/data/home/Xetd71/Content-based-Neural-Recommender-Systems/models/mlp


## Load data

In [4]:
PREPROC_DIR = f'{DATA_DIR}preproc/'

In [5]:
# users
users_factors = np.load(f'{PREPROC_DIR}users_factors.npy')
users_mean_ratings = np.load(f'{PREPROC_DIR}users_mean_ratings.npy')
print(users_factors.shape, users_mean_ratings.shape)

# items
items_factors = np.load(f'{PREPROC_DIR}items_factors.npy')
items_processed_text = np.load(f'{PREPROC_DIR}items_processed_text.npy')
items_processed_images = np.load(f'{PREPROC_DIR}items_processed_images.npy')
print(items_factors.shape, items_processed_text.shape, items_processed_images.shape)

# ratings
ratings_matrix = np.load(f'{PREPROC_DIR}ratings_matrix.npy')
ratings_0_matrix = np.load(f'{PREPROC_DIR}ratings_0_matrix.npy')
ratings_1_matrix = np.load(f'{PREPROC_DIR}ratings_1_matrix.npy')
print(ratings_matrix.shape, ratings_0_matrix.shape, ratings_1_matrix.shape)

(42977, 64) (42977, 1)
(328050, 64) (328050, 96) (328050, 96)
(67780168, 3) (61329513, 3) (6450655, 3)


In [6]:
w1 = ratings_1_matrix.shape[0]/ratings_matrix.shape[0]
w0 = ratings_0_matrix.shape[0]/ratings_matrix.shape[0]

In [7]:
w0, w1

(0.9048297578725387, 0.09517024212746124)

In [8]:
w1 = 67780168/2/6450655
w0 = 67780168/2/(67780168-6450655)
w1, w0

(5.253743069502244, 0.5525901371497928)

In [9]:
w0/w1

0.10518027429958558

In [28]:
w0, w1 = 0.75, 0.25

In [10]:
6450655/67780168, 1-6450655/67780168

(0.09517024212746124, 0.9048297578725387)

In [11]:
items_matrix = np.concatenate([items_processed_text, items_processed_images], axis=1)
items_matrix.shape

(328050, 192)

In [12]:
ratings_matrix[ratings_matrix[:, 0] < 3]

array([[     0, 206495,      0],
       [     0, 279694,      0],
       [     0,  19718,      0],
       ...,
       [     2,  50780,      0],
       [     2, 192450,      0],
       [     2,  53749,      0]])

In [100]:
def ratings_batch(ratings_matrix=ratings_matrix, batch_size=64):
    idxs = np.random.randint(0, ratings_matrix.shape[0], batch_size)
    items = ratings_matrix[idxs]
    return items[:, 0], items_matrix[items[:, 1]], items[:, 2]

def normalized_ratings_batch(batch_size=64):
    return tuple(map(lambda x: np.concatenate((x[0], x[1]), axis=0), zip(
        ratings_batch(ratings_0_matrix, batch_size // 2), 
        ratings_batch(ratings_1_matrix, batch_size - batch_size // 2)
    )))

## Train model

In [114]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.callbacks import TensorBoard

In [115]:
# reset graph when you change architecture!
def reset_tf_session():
    curr_session = tf.get_default_session()
    # close current session
    if curr_session is not None:
        curr_session.close()
    # reset graph
    K.clear_session()
    # create new session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    K.set_session(s)
    return s

In [116]:
# import necessary building blocks
from keras.models import Model
from keras.layers import Dense, Input, Embedding, concatenate, Reshape, Dropout

In [117]:
N_USERS = 42977
USER_EMBEDDING_SIZE = 96
ITEM_EMBEDDING_SIZE = 192

In [118]:
def make_model():
    user_id = Input(dtype='int32', shape=(1,), name='user_id')
    item_embedding = Input(dtype='float32', shape=(ITEM_EMBEDDING_SIZE,), name='item')    
#     y = tf.placeholder(dtype=tf.float32, shape=[None], name='y')
    
    user_embedding = Reshape((USER_EMBEDDING_SIZE,))(Embedding(N_USERS, USER_EMBEDDING_SIZE, input_length=1)(user_id))
    
    layer = concatenate([user_embedding, item_embedding])
    layer = Dense(512, activation="elu", input_shape=(288,))(layer)
#     layer = Dense(512, activation="elu", input_shape=(512,))(layer)
    layer = Dense(512, activation="elu", input_shape=(512,))(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(128, activation="elu", input_shape=(512,))(layer)
    pred_y = Dense(1, activation="sigmoid", input_shape=(64,))(layer)
    
    model = Model(inputs=[user_id, item_embedding], outputs=pred_y)
    
    return model

In [119]:
# describe model
s = reset_tf_session()  # clear default graph
model = make_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 96)        4125792     user_id[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 96)           0           embedding_1[0][0]                
__________________________________________________________________________________________________
item (InputLayer)               (None, 192)          0                                            
__________________________________________________________________________________________________
concatenat

In [136]:
BATCH_SIZE = 512
STEPS_PER_EPOCH = 40_000
EPOCHS = 50

s = reset_tf_session()  # clear default graph
model = make_model()  # define our model

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.adam(lr=0.001),  # gradient clipping just in case
)

In [137]:
# for saving the model after every epoch
from keras.models import save_model

class ModelSaveCallback(keras.callbacks.Callback):
    def __init__(self, file_name, period):
        super(ModelSaveCallback, self).__init__()
        self.file_name = file_name
        self.period = period

    def on_epoch_end(self, epoch, logs):
        model_filename = self.file_name.format(epoch)
        if (epoch + 1) % self.period == 0:
            save_model(self.model, model_filename)
            print(f"Model saved in {model_filename}")

In [138]:
def mlp_predict(user_id, items):
    return model.predict([np.full(len(items), user_id),  items_matrix[items]])[:, 0]

def prediction_hist(p, bins=5):
    hist = []
    for th in np.linspace(0, 1-1/bins, bins):
        hist.append(np.logical_and(th <= p, p <= (th+1/bins)).sum())
    return hist

def mlp_get_prediction_hist(bins=5):
    user_id, item_embeddings, ratings = normalized_ratings_batch(batch_size=BATCH_SIZE)
    return prediction_hist(model.predict([user_id, item_embeddings])[:, 0], bins)

In [139]:
# for saving the model after every epoch
from keras.models import save_model

class ModelLogFileCallback(keras.callbacks.Callback):
    def __init__(self, file_name):
        super(ModelLogFileCallback, self).__init__()
        self.file_name = file_name
        
    def on_train_begin(self, logs):
        self.file = open(self.file_name, 'a+')
        self.epoch_time = time.time()
 
    def on_train_end(self, logs):
        self.file.close()

    def on_epoch_end(self, epoch, logs):
        ndcg = test.ndcg(mlp_predict)
        p_hist = mlp_get_prediction_hist()
        cur_time = time.time()
        logs['ndcg'] = ndcg
        msg = "Epoch {},   loss: {:.4f},   val ndcg: {:.4f},   p_hist: [{}],   time: {:.4f}".format(
            epoch,
            logs.get('loss'),
            ndcg,
            ', '.join(list(map(str, p_hist))),
            cur_time - self.epoch_time,
        )
        print(msg)
        print(msg, file=self.file, flush=True)
        self.epoch_time = cur_time

In [140]:
last_finished_epoch = 0

# you can continue from snapshot!!!
# from keras.models import load_model
# s = reset_tf_session()
# last_finished_epoch = 4
# model = load_model("/data/home/Xetd71/MLP/model_{}".format(last_finished_epoch))

In [141]:
def train_iterator(batch_size):
    while True:
        user_id, item_embeddings, ratings = normalized_ratings_batch(batch_size=batch_size)
#         user_id, item_embeddings, ratings = ratings_batch(batch_size=batch_size, ratings_matrix=ratings_matrix)
        yield [user_id, item_embeddings], ratings 

In [142]:
import time
tensorboard = TensorBoard(log_dir=f"/data/home/Xetd71/MLP/logs/{time.time()}")

In [143]:
# fit the model with our eternal generator!
model.fit_generator(
    train_iterator(BATCH_SIZE), 
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS,
    callbacks=[
        ModelSaveCallback("/data/home/Xetd71/MLP/model_{}", 1),
        ModelLogFileCallback("/data/home/Xetd71/MLP/logs.txt"),
        tensorboard,
    ],
    verbose=1,
    initial_epoch=last_finished_epoch,
#     class_weight={0: 1/w0/2, 1: 1/w1/2}
)

Epoch 1/50
40000/40000 [==============================] - 377s 9ms/step - loss: 0.6192
Model saved in /data/home/Xetd71/MLP/model_0
Epoch 0,   loss: 0.6192,   val ndcg: 0.2119,   p_hist: [72, 125, 160, 125, 30],   time: 477.3278
Epoch 2/50
40000/40000 [==============================] - 378s 9ms/step - loss: 0.6023
Model saved in /data/home/Xetd71/MLP/model_1
Epoch 1,   loss: 0.6023,   val ndcg: 0.2259,   p_hist: [45, 123, 152, 159, 33],   time: 478.8193
Epoch 3/50
40000/40000 [==============================] - 378s 9ms/step - loss: 0.5896
Model saved in /data/home/Xetd71/MLP/model_2
Epoch 2,   loss: 0.5896,   val ndcg: 0.2329,   p_hist: [65, 115, 119, 167, 46],   time: 478.7584
Epoch 4/50
40000/40000 [==============================] - 379s 9ms/step - loss: 0.5782
Model saved in /data/home/Xetd71/MLP/model_3
Epoch 3,   loss: 0.5782,   val ndcg: 0.2373,   p_hist: [68, 127, 138, 125, 54],   time: 478.9073
Epoch 5/50
40000/40000 [==============================] - 378s 9ms/step - loss: 0.56

KeyboardInterrupt: 

In [16]:
N_USERS = 42977
USER_EMBEDDING_SIZE = 96
ITEM_EMBEDDING_SIZE = 192
EPSILON = 1e-6
MEAN_RATING = 0.09517
# 0.09517

LR = 0.0001
BATCH_SIZE = 512
EPOCHS = 100
N_BATCHES_PER_EPOCH = 100_000

In [17]:
class MLP:
    user_id = tf.placeholder(dtype=tf.int32, shape=[None], name='user_id')
    item_embedding = tf.placeholder( tf.float32, shape=[None, ITEM_EMBEDDING_SIZE], name='item_embedding')
    y = tf.placeholder(dtype=tf.float32, shape=[None], name='y')
    
    user_embeddings = tf.Variable(tf.random_normal([N_USERS, USER_EMBEDDING_SIZE]), dtype=tf.float32)
    user_embedding = tf.nn.embedding_lookup(user_embeddings, user_id)
    
    layer = tf.concat([user_embedding, item_embedding], axis=1)
    layer = L.Dense(512, "elu", input_shape=(None, 288))(layer)
    layer = L.Dense(512, "elu", input_shape=(None, 512))(layer)
    layer = L.Dense(512, "elu", input_shape=(None, 512))(layer)
    layer = L.Dense(128, "elu", input_shape=(None, 512))(layer)
    layer = L.Dense(64, "elu", input_shape=(None, 128))(layer)
    pred_y = L.Dense(1, "sigmoid", input_shape=(None, 64))(layer)
    pred_y = tf.clip_by_value(pred_y, EPSILON, 1 - EPSILON)

    loss = - tf.reduce_mean(y * tf.log(pred_y) *w1 + (1 - y) * tf.log(1 - pred_y) * w0)

NameError: name 'L' is not defined

In [18]:
# define optimizer operation to minimize the loss
optimizer = tf.train.AdamOptimizer(learning_rate=LR)
train_step = optimizer.minimize(MLP.loss)

# will be used to save/load network weights.
# you need to reset your default graph and define it in the same way to be able to load the saved weights!
saver = tf.train.Saver()

# intialize all variables
sess.run(tf.global_variables_initializer())

NameError: name 'MLP' is not defined

In [32]:
total_parameters = 0
for variable in tf.trainable_variables():
    shape = variable.get_shape()
    variable_parameters = 1
    for dim in shape:
        variable_parameters *= dim.value
    total_parameters += variable_parameters
print(total_parameters)

4873057


In [33]:
def mlp_predict(user_id, items):
    return sess.run(
            [MLP.pred_y],
            feed_dict={
                MLP.user_id: np.full(len(items), user_id),
                MLP.item_embedding: items_matrix[items]}
    )[0][:, 0]

In [34]:
def prediction_hist(p, bins=5):
    hist = []
    for th in np.linspace(0, 1-1/bins, bins):
        hist.append(np.logical_and(th <= p, p <= (th+1/bins)).sum())
    return hist

In [35]:
!pwd

/data/home/Xetd71/Content-based-Neural-Recommender-Systems/models/mlp


In [36]:
# saver = tf.train.import_meta_graph('/data/home/Xetd71/MLP/mlp45.meta')
# saver.restore(sess,tf.train.latest_checkpoint('/data/home/Xetd71/MLP/mlp45'))
# graph = tf.get_default_graph()
# loss = graph.get_tensor_by_name("Neg:0") #Tensor to import

# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# train_step = optimizer.minimize(loss)

In [37]:
ratings_matrix=ratings_matrix[ratings_matrix[:, 0] < 3]

In [38]:
# to make training reproducible
np.random.seed(42)
log_file = open('/data/home/Xetd71/MLP/logs.txt', 'a+')
for epoch in range(EPOCHS):
    train_loss = 0
    p_hist = np.array([])
    pbar = tqdm_utils.tqdm_notebook_failsafe(range(N_BATCHES_PER_EPOCH))
    counter = 0
    for i, _ in enumerate(pbar):
        user_ids, items, ratings = ratings_batch(batch_size=BATCH_SIZE)
        batch_train_loss, batch_train_p, _ = sess.run(
            [MLP.loss, MLP.pred_y,  train_step],
            feed_dict={MLP.user_id: user_ids, MLP.item_embedding: items, MLP.y: ratings}
        )
        train_loss += batch_train_loss
        counter += 1
        p_hist = prediction_hist(batch_train_p)
        pbar.set_description("Training loss: {:.4f}; hist: [{}]".format(train_loss / counter, ', '.join(list(map(str, p_hist)))))

        
    train_loss /= N_BATCHES_PER_EPOCH
    epoch_msg = 'Epoch: {}, train loss: {:.4f}, val ndcg: {:.4f}'.format(epoch, train_loss, test.ndcg(mlp_predict))
    print(epoch_msg)
    print(epoch_msg, file=log_file, flush=True)
    if epoch % 5 == 0:
        saver.save(sess, f"/data/home/Xetd71/MLP/mlp{epoch}")
    
log_file.close()
print("Finished!")

KeyboardInterrupt: 